In [1]:
from IPython import parallel as p
import ProgressBar as pb

In [2]:
pc = p.Client()

In [3]:
pc.ids

[0, 1, 2, 3, 4, 5]

In [4]:
view = pc.load_balanced_view()

In [5]:
%px import numpy as np
%px import pandas as pd
%px dates = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/timecode.pcl')
%px wyears = np.load('/Volumes/Users/Theo/projects/Budyko_vic/wyears.npy')
%px forcing_columns = ['P','Tmax','Tmin','W']

In [6]:
import pandas as pd
import numpy as np
import glob

dates = pd.read_pickle('./timecode.pcl')
wyears = np.load('./wyears.npy')
forcing_columns = ['P','Tmax','Tmin','W']

In [26]:
def meantemp(fr,idx):
    lat = float(fr.split('_')[-2]) # extract latitude from file name
    lon = float(fr.split('_')[-1]) # extract longitude from file name

    force = pd.read_table(fr, sep=' ', names=forcing_columns)
    force.index = pd.DatetimeIndex(dates)
        
    return lat,lon,idx,round((force.Tmin.mean()+force.Tmax.mean())/2,3)

In [8]:
files = pd.read_pickle('./forcing_fluxes_filenames_lat_lon_index.df')

In [9]:
files.head()

,exists,flux,forcing,lat,lon,indexer
0,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.03125,1
1,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.09375,2
2,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.15625,3
3,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.21875,4
4,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.28125,5


In [27]:
%time meantemp(files.forcing[270000],files.indexer[270000])

CPU times: user 12.9 ms, sys: 4.2 ms, total: 17.1 ms
Wall time: 11.8 s


(50.21875, -118.53125, 270001, 4.829)

In [28]:
res = view.map(meantemp,files.forcing[270000:275000],files.indexer[270000:275000])

In [ ]:
prog = pb.ProgressBar(len(files.forcing[270000:275000]))

while res.ready() == False:
    prog.animate(res.progress)

In [33]:
res = view.map(meantemp,files.forcing,files.indexer)

In [34]:
prog = pb.ProgressBar(len(files.forcing))

while res.ready() == False:
    prog.animate(res.progress)

[****************100%******************]  309558 of 309673 complete


In [35]:
res.ready()

True

In [36]:
res.result

[(25.03125, -100.03125, 1, 18.889),
 (25.03125, -100.09375, 2, 18.872),
 (25.03125, -100.15625, 3, 16.202),
 (25.03125, -100.21875, 4, 13.401),
 (25.03125, -100.28125, 5, 11.491),
 (25.03125, -100.34375, 6, 9.998),
 (25.03125, -100.40625, 7, 11.63),
 (25.03125, -100.46875, 8, 14.103),
 (25.03125, -100.53125, 9, 15.459),
 (25.03125, -100.59375, 10, 15.736),
 (25.03125, -100.65625, 11, 15.877),
 (25.03125, -100.71875, 12, 16.034),
 (25.03125, -100.78125, 13, 16.036),
 (25.03125, -100.84375, 14, 15.464),
 (25.03125, -100.90625, 15, 14.975),
 (25.03125, -100.96875, 16, 14.887),
 (25.03125, -101.03125, 17, 15.768),
 (25.03125, -101.09375, 18, 15.896),
 (25.03125, -101.15625, 19, 15.927),
 (25.03125, -101.21875, 20, 16.315),
 (25.03125, -101.28125, 21, 15.32),
 (25.03125, -101.34375, 22, 15.881),
 (25.03125, -101.40625, 23, 15.275),
 (25.03125, -101.46875, 24, 14.672),
 (25.03125, -101.53125, 25, 15.586),
 (25.03125, -101.59375, 26, 16.462),
 (25.03125, -101.65625, 27, 17.406),
 (25.03125, -

In [37]:
lat,lon,idx,Tmean = zip(*res.result)

In [41]:
np.savez_compressed('./mean_temp.npz',lat=lat,lon=lon,idx=idx,Tmean=Tmean)

In [40]:
(res.elapsed/60./60.)

0.523523705